In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

Using TensorFlow backend.
c:\users\hp\anaconda3\envs\vks_test\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\hp\anaconda3\envs\vks_test\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\hp\anaconda3\envs\vks_test\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\hp\anaconda3\envs\vks_test\lib\site-pa

In [2]:
IMAGE_SIZE = [224, 224]

In [3]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
vgg

In [4]:
#Give dataset path
train_path = 'E:/data/train'
test_path = 'E:/data/train'

In [5]:
vgg.layers

In [6]:
# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False

In [7]:
# useful for getting number of classes
folders = glob('E:/data/train/*')
print(len(folders))

4


In [8]:
vgg.output

<tf.Tensor 'block5_pool/MaxPool:0' shape=(?, 7, 7, 512) dtype=float32>

In [9]:
#The Original Image Size
vgg.input

<tf.Tensor 'input_1:0' shape=(?, 224, 224, 3) dtype=float32>

In [10]:
x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [11]:
from keras import optimizers


sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9)
model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [12]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [13]:
# Data Augmentation
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [14]:
# Make sure you provide the same target size as initialied for the image size
train_set = train_datagen.flow_from_directory('E:/data/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 475 images belonging to 4 classes.


In [15]:
test_set = test_datagen.flow_from_directory('E:/data/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 264 images belonging to 4 classes.


In [16]:
from datetime import datetime
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau

#lr_scheduler = LearningRateScheduler(lr_schedule)

#lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
#                               cooldown=0,
#                               patience=5,
#                               min_lr=0.5e-6)

#num_epochs = 1000
#num_batch_size = 32

checkpoint = ModelCheckpoint(filepath='mymodel.h5', 
                               verbose=1, save_best_only=True)

callbacks = [checkpoint]

start = datetime.now()

model.fit_generator(
  train_set,
  validation_data=test_set,
  epochs=2,
  steps_per_epoch=5,
  validation_steps=32,
    callbacks=callbacks ,verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/2


c:\users\hp\anaconda3\envs\vks_test\lib\site-packages\PIL\Image.py:961: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


5/5 [==============================] - 587s 117s/step - loss: 2.8301 - accuracy: 0.7781 - val_loss: 2.8453 - val_accuracy: 0.8072

Epoch 00001: val_loss improved from inf to 2.84527, saving model to mymodel.h5
Epoch 2/2
5/5 [==============================] - 528s 106s/step - loss: 2.6623 - accuracy: 0.8250 - val_loss: 3.0241 - val_accuracy: 0.7909

Epoch 00002: val_loss did not improve from 2.84527
Training completed in time:  0:18:37.376421


In [17]:
from keras.models import load_model
model= load_model("mymodel.h5")
img = cv2.imread('E:/data/test/footwear/0000408.jpg')
dim =(224,224)
img = cv2.resize(img,dim,interpolation=cv2.INTER_AREA)
x = image.img_to_array(img)
x = np.expand_dims(x,axis=0)
x = preprocess_input(x)
preds = model.predict(x)




NameError: name 'cv2' is not defined

In [18]:
model.save("mymodel.h5")
print("Saved model to disk")

Saved model to disk


In [19]:
import numpy as np

from keras.preprocessing import image

In [20]:
from tensorflow.python.keras.models import load_model

In [25]:
test_image = image.load_img('E:/data/test/footwear/00000408.jpg', target_size = (224, 224))

In [28]:
test_image = image.img_to_array(test_image)

In [29]:
test_image = np.expand_dims(test_image, axis = 0)

In [30]:
model = load_model('mymodel.h5')

In [31]:
result = model.predict(test_image)

In [32]:
result = result[0]

In [33]:
classes = ['backpack','footwear', 'glasses', 'watch']

label_name = {classes[i]: result[i] for i in range(len(result))}
label_name

{'backpack': 0.0, 'footwear': 1.0, 'glasses': 0.0, 'watch': 0.0}

In [34]:
output = max(label_name, key=label_name.get)
print(output)

footwear
